In [ ]:
%pip install tensorflow
%pip install transformers

In [ ]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel,

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = TFAutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [ ]:
text = "ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد می‌توانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)


In [ ]:
model.config

In [ ]:
text = "ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد می‌توانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors="tf")

In [ ]:
import tensorflow as tf
input_ids = tf.convert_to_tensor(encoded_input["input_ids"].numpy())

In [ ]:
attention_mask = tf.convert_to_tensor(encoded_input["attention_mask"].numpy())

In [ ]:
token_type_ids = tf.convert_to_tensor(encoded_input["token_type_ids"].numpy())

In [ ]:
output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

In [ ]:
output = model(encoded_input)

In [ ]:
import numpy as np
token_embeddings = output.last_hidden_state
for i in range(token_embeddings.shape[1]):
    token_id = encoded_input["input_ids"][0][i].numpy()
    token = tokenizer.convert_ids_to_tokens(np.array([token_id]))
    embedding = token_embeddings[0][i].numpy()
    print(f"Token: {token}")
    print(f"Embedding: {embedding}")

In [ ]:
import numpy as np
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

# Prepare the training data
train_sentences = ["این یک مثال است", "مثال دیگری است"]
train_labels = [
    ["DET", "NUM", "NOUN", "VERB", "PUNCT"],
    ["NOUN", "ADJ", "VERB", "DET", "ADJ", "NOUN", "PUNCT"],
]

train_tokenized = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="tf")
train_input_ids = train_tokenized["input_ids"].numpy()
train_attention_mask = train_tokenized["attention_mask"]
train_labels_encoded = np.array([[tokenizer.convert_tokens_to_ids(label) for label in labels] for labels in train_labels])

# Define the model architecture
num_labels = len(tokenizer.get_vocab())
model = TFBertForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", num_labels=num_labels)

# Trainthe model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
model.fit(
    x=(train_input_ids, train_attention_mask),
    y=train_labels_encoded,
    epochs=3
)

# Test the model
test_sentence = "این یک مثال دیگر است"
test_tokenized = tokenizer(test_sentence, padding=True, truncation=True, return_tensors="tf")
test_input_ids = test_tokenized["input_ids"]
test_attention_mask = test_tokenized["attention_mask"]
test_logits = model.predict((test_input_ids, test_attention_mask))[0]
test_labels = tf.argmax(test_logits, axis=-1).numpy()
test_tokens = tokenizer.convert_ids_to_tokens(test_input_ids[0])
test_reordered_tokens = [test_tokens[i] for i in test_labels[1:-1]]
test_reordered_sentence = tokenizer.convert_tokens_to_string(test_reordered_tokens)
print(test_reordered_sentence)